## 1. QUES dataset

In [1]:
import numpy as np
import pandas as pd

In [3]:
data_train = pd.read_csv("D:/UROP/datasets/QUES.csv")
data_train.head()

,DIT,NOC,mpc,rfc,lcom,dac,wmc,nom,size2,size1,change
0,0,0,2,27,5,1,38,24,25,365,102
1,2,0,18,39,3,2,4,5,9,174,85
2,2,0,11,29,4,1,2,5,7,122,38
3,2,0,20,96,26,5,28,31,36,485,81
4,2,0,8,62,10,8,31,29,37,310,55


In [4]:
data_train.shape

(71, 11)

In [5]:
Y_train = pd.DataFrame(data = data_train.iloc[:, -1].values, columns = ['target'])
data_train.head()

,DIT,NOC,mpc,rfc,lcom,dac,wmc,nom,size2,size1,change
0,0,0,2,27,5,1,38,24,25,365,102
1,2,0,18,39,3,2,4,5,9,174,85
2,2,0,11,29,4,1,2,5,7,122,38
3,2,0,20,96,26,5,28,31,36,485,81
4,2,0,8,62,10,8,31,29,37,310,55


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_train, Y_train, test_size=0.2, random_state=42)
X_train.shape

(56, 11)

In [7]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.fit_transform(X_test)
X_train.mean()

0.20439392769858392

In [8]:
from sklearn import metrics
def print_error(X_test, y_test, model_name):
    prediction = model_name.predict(X_test)
    print('Mean Squared Error:', metrics.mean_squared_error(y_test, prediction))
    print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, prediction)))

### Linear Regression

In [9]:
from sklearn import linear_model
linear_regression = linear_model.LinearRegression()
linear_regression.fit(X_train, y_train)
print_error(X_test,y_test, linear_regression)

Mean Squared Error: 3770.4290669441525
Root Mean Squared Error: 61.40381964458036


In [10]:
from sklearn import linear_model
linear_regression = linear_model.LinearRegression()
linear_regression.fit(X_train, y_train)
MRE = []
prediction = linear_regression.predict(X_test)
prediction = prediction.reshape(15,1)

MRE.append(abs(y_test-prediction)/y_test)

mre = str(MRE).split()
mre.remove("target")
mre_new = mre[2::2]
mre_new[len(mre_new)-1] = mre_new[len(mre_new)-1].strip("]")
for i in range(len(mre_new)):
    mre_new[i] = (float(mre_new[i]))

MMRE = sum(mre_new)/len(y_test)
print(MMRE)
count=0
for i in range(len(mre_new)):
    if(mre_new[i]<=0.3):
        count+=1
p=count/len(mre_new)
print("Pred(0.3) is", p)

0.8396298666666665
Pred(0.3) is 0.06666666666666667


### SVR

In [11]:
from sklearn.svm import SVR
supportvector_regressor = SVR()
supportvector_regressor.fit(X_train,y_train)
print_error(X_test,y_test, supportvector_regressor)

Mean Squared Error: 829.6748571717144
Root Mean Squared Error: 28.80407709286507


D:\anaconda\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [12]:
from sklearn.svm import SVR
model = SVR()
model.fit(X_train,y_train)
print_error(X_test,y_test, supportvector_regressor)

MRE = []
prediction = model.predict(X_test)
prediction = prediction.reshape(15,1)
MRE.append(abs(y_test-prediction)/y_test)

mre = str(MRE).split()
mre.remove("target")
mre_new = mre[2::2]
mre_new[len(mre_new)-1] = mre_new[len(mre_new)-1].strip("]")
for i in range(len(mre_new)):
    mre_new[i] = (float(mre_new[i]))
MMRE = sum(mre_new)/len(y_test)
print("MMRE is: ",MMRE)
count=0
for i in range(len(mre_new)):
    if(mre_new[i]<=0.3):
        count+=1
p=count/len(mre_new)
print("Pred(0.3) is", p)

Mean Squared Error: 829.6748571717144
Root Mean Squared Error: 28.80407709286507
MMRE is:  1.079774266666667
Pred(0.3) is 0.3333333333333333


D:\anaconda\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


### ANN

In [64]:
import keras
from keras.layers import Dense
import tensorflow as tf
ann = keras.models.Sequential([
    Dense(6, activation="relu",
          input_shape=X_train.shape[1:]),
    Dense(6,activation="relu"),
    Dense(1)
])
optimizer=tf.keras.optimizers.Adam()
loss=keras.losses.mean_squared_error
ann.compile(optimizer=optimizer,loss=loss,metrics=["mean_squared_error"])

history=ann.fit(X_train,y_train,epochs=100)
ann.summary()
print_error(X_test,y_test,ann)

Epoch 1/100
2/2 [==============================] - 0s 6ms/step - loss: 6659.9932 - mean_squared_error: 6659.9932
Epoch 2/100
2/2 [==============================] - 0s 5ms/step - loss: 6659.5459 - mean_squared_error: 6659.5459
Epoch 3/100
2/2 [==============================] - 0s 5ms/step - loss: 6659.0981 - mean_squared_error: 6659.0981
Epoch 4/100
2/2 [==============================] - 0s 5ms/step - loss: 6658.6445 - mean_squared_error: 6658.6445
Epoch 5/100
2/2 [==============================] - 0s 5ms/step - loss: 6658.2153 - mean_squared_error: 6658.2153
Epoch 6/100
2/2 [==============================] - 0s 4ms/step - loss: 6657.6851 - mean_squared_error: 6657.6851
Epoch 7/100
2/2 [==============================] - 0s 5ms/step - loss: 6657.1528 - mean_squared_error: 6657.1528
Epoch 8/100
2/2 [==============================] - 0s 4ms/step - loss: 6656.5312 - mean_squared_error: 6656.5312
Epoch 9/100
2/2 [==============================] - 0s 5ms/step - loss: 6655.9194 - mean_squared_

In [65]:
import keras
from keras.layers import Dense
import tensorflow as tf
ann = keras.models.Sequential([
    Dense(6, activation="relu",
          input_shape=X_train.shape[1:]),
    Dense(6,activation="relu"),
    Dense(1)
])
optimizer=tf.keras.optimizers.Adam()
loss=keras.losses.mean_squared_error
ann.compile(optimizer=optimizer,loss=loss,metrics=["mean_squared_error"])

history=ann.fit(X_train,y_train,epochs=100)
ann.summary()
print_error(X_test,y_test, linear_regression)

MRE = []
prediction = model.predict(X_test)
prediction = prediction.reshape(15,1)
MRE.append(abs(y_test-prediction)/y_test)

mre = str(MRE).split()
mre.remove("target")
mre_new = mre[2::2]
mre_new[len(mre_new)-1] = mre_new[len(mre_new)-1].strip("]")
for i in range(len(mre_new)):
    mre_new[i] = (float(mre_new[i]))
MMRE = sum(mre_new)/len(y_test)
print("MMRE is: ",MMRE)
count=0
for i in range(len(mre_new)):
    if(mre_new[i]<=0.3):
        count+=1
p=count/len(mre_new)
print("Pred(0.3) is", p)

Epoch 1/100
2/2 [==============================] - 1s 6ms/step - loss: 6658.4360 - mean_squared_error: 6658.4360
Epoch 2/100
2/2 [==============================] - 0s 11ms/step - loss: 6657.4878 - mean_squared_error: 6657.4878
Epoch 3/100
2/2 [==============================] - 0s 13ms/step - loss: 6656.6060 - mean_squared_error: 6656.6060
Epoch 4/100
2/2 [==============================] - 0s 5ms/step - loss: 6655.6284 - mean_squared_error: 6655.6284
Epoch 5/100
2/2 [==============================] - 0s 5ms/step - loss: 6654.5391 - mean_squared_error: 6654.5391
Epoch 6/100
2/2 [==============================] - 0s 4ms/step - loss: 6653.5073 - mean_squared_error: 6653.5073
Epoch 7/100
2/2 [==============================] - 0s 5ms/step - loss: 6652.4399 - mean_squared_error: 6652.4399
Epoch 8/100
2/2 [==============================] - 0s 10ms/step - loss: 6651.3350 - mean_squared_error: 6651.3350
Epoch 9/100
2/2 [==============================] - 0s 7ms/step - loss: 6650.2031 - mean_squar

## 2. UIMS dataset

In [14]:
data_train = pd.read_csv("D:/UROP/datasets/UIMS1.csv")
data_train.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,1,3,4,14,5,1,3,7,9,37,14
1,3,0,6,20,6,2,5,7,9,94,18
2,2,0,1,12,4,1,1,6,7,26,2
3,2,0,1,2,1,0,1,1,1,4,2
4,1,0,1,21,12,8,0,18,26,60,10


In [15]:
data_train.shape

(39, 11)

In [16]:
Y_train = pd.DataFrame(data = data_train.iloc[:, -1].values, columns = ['target'])
data_train.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,1,3,4,14,5,1,3,7,9,37,14
1,3,0,6,20,6,2,5,7,9,94,18
2,2,0,1,12,4,1,1,6,7,26,2
3,2,0,1,2,1,0,1,1,1,4,2
4,1,0,1,21,12,8,0,18,26,60,10


In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_train, Y_train, test_size=0.2, random_state=42)
X_train.shape

(31, 11)

In [18]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.fit_transform(X_test)
X_train.mean()

0.24461729148947517

In [28]:
from sklearn import metrics
def print_error(X_test, y_test, model_name):
    prediction = model_name.predict(X_test)
    print('Mean Squared Error:', metrics.mean_squared_error(y_test, prediction))
    print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, predict)))

### Linear Regression

In [20]:
from sklearn import linear_model
linear_regression = linear_model.LinearRegression()
linear_regression.fit(X_train, y_train)
print_error(X_test,y_test, linear_regression)

Mean Squared Error: 170.70414755321428
Root Mean Squared Error: 13.06537973245379


In [21]:
from sklearn import linear_model
linear_regression = linear_model.LinearRegression()
linear_regression.fit(X_train, y_train)
MRE = []
prediction = linear_regression.predict(X_test)
prediction = prediction.reshape(8,1)

MRE.append(abs(y_test-prediction)/y_test)

mre = str(MRE).split()
mre.remove("target")
mre_new = mre[2::2]
mre_new[len(mre_new)-1] = mre_new[len(mre_new)-1].strip("]")
for i in range(len(mre_new)):
    mre_new[i] = (float(mre_new[i]))

MMRE = sum(mre_new)/len(y_test)
print(MMRE)
count=0
for i in range(len(mre_new)):
    if(mre_new[i]<=0.3):
        count+=1
p=count/len(mre_new)
print("Pred(0.3) is", p)

0.09754985000001203
Pred(0.3) is 1.0


### SVR

In [22]:
from sklearn.svm import SVR
supportvector_regressor = SVR()
supportvector_regressor.fit(X_train,y_train)
print_error(X_test,y_test, supportvector_regressor)

Mean Squared Error: 6731.318724341526
Root Mean Squared Error: 82.04461423117014


D:\anaconda\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [24]:
from sklearn.svm import SVR
model = SVR()
model.fit(X_train,y_train)
print_error(X_test,y_test, supportvector_regressor)

MRE = []
prediction = model.predict(X_test)
prediction = prediction.reshape(8,1)
MRE.append(abs(y_test-prediction)/y_test)

mre = str(MRE).split()
mre.remove("target")
mre_new = mre[2::2]
mre_new[len(mre_new)-1] = mre_new[len(mre_new)-1].strip("]")
for i in range(len(mre_new)):
    mre_new[i] = (float(mre_new[i]))
MMRE = sum(mre_new)/len(y_test)
print("MMRE is: ",MMRE)
count=0
for i in range(len(mre_new)):
    if(mre_new[i]<=0.3):
        count+=1
p=count/len(mre_new)
print("Pred(0.3) is", p)

Mean Squared Error: 6731.318724341526
Root Mean Squared Error: 82.04461423117014
MMRE is:  2.4675267499999998
Pred(0.3) is 0.25


D:\anaconda\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


### ANN

In [77]:
import keras
from keras.layers import Dense
import tensorflow as tf
ann = keras.models.Sequential([
    Dense(6, activation="relu",
          input_shape=X_train.shape[1:]),
    Dense(6,activation="relu"),
    Dense(1)
])
optimizer=tf.keras.optimizers.Adam()
loss=keras.losses.mean_squared_error
ann.compile(optimizer=optimizer,loss=loss,metrics=["mean_squared_error"])

history=ann.fit(X_train,y_train,epochs=2000)
ann.summary()
print_error(X_test,y_test,ann)

Epoch 1/2000
1/1 [==============================] - 1s 941ms/step - loss: 6861.7729 - mean_squared_error: 6861.7729
Epoch 2/2000
1/1 [==============================] - 0s 11ms/step - loss: 6860.2085 - mean_squared_error: 6860.2085
Epoch 3/2000
1/1 [==============================] - 0s 11ms/step - loss: 6858.6401 - mean_squared_error: 6858.6401
Epoch 4/2000
1/1 [==============================] - 0s 19ms/step - loss: 6857.0645 - mean_squared_error: 6857.0645
Epoch 5/2000
1/1 [==============================] - 0s 13ms/step - loss: 6855.4829 - mean_squared_error: 6855.4829
Epoch 6/2000
1/1 [==============================] - 0s 12ms/step - loss: 6853.8916 - mean_squared_error: 6853.8916
Epoch 7/2000
1/1 [==============================] - 0s 9ms/step - loss: 6852.2949 - mean_squared_error: 6852.2949
Epoch 8/2000
1/1 [==============================] - 0s 9ms/step - loss: 6850.6914 - mean_squared_error: 6850.6914
Epoch 9/2000
1/1 [==============================] - 0s 8ms/step - loss: 6849.0854

In [78]:
import keras
from keras.layers import Dense
import tensorflow as tf
ann = keras.models.Sequential([
    Dense(6, activation="relu",
          input_shape=X_train.shape[1:]),
    Dense(6,activation="relu"),
    Dense(1)
])
optimizer=tf.keras.optimizers.Adam()
loss=keras.losses.mean_squared_error
ann.compile(optimizer=optimizer,loss=loss,metrics=["mean_squared_error"])

history=ann.fit(X_train,y_train,epochs=2000)
ann.summary()
print_error(X_test,y_test,ann)
MRE = []
prediction = model.predict(X_test)
prediction = prediction.reshape(8,1)
MRE.append(abs(y_test-prediction)/y_test)

mre = str(MRE).split()
mre.remove("target")
mre_new = mre[2::2]
mre_new[len(mre_new)-1] = mre_new[len(mre_new)-1].strip("]")
for i in range(len(mre_new)):
    mre_new[i] = (float(mre_new[i]))
MMRE = sum(mre_new)/len(y_test)
print("MMRE is: ",MMRE)
count=0
for i in range(len(mre_new)):
    if(mre_new[i]<=0.3):
        count+=1
p=count/len(mre_new)
print("Pred(0.3) is", p)

Epoch 1/2000
1/1 [==============================] - 1s 576ms/step - loss: 6856.6445 - mean_squared_error: 6856.6445
Epoch 2/2000
1/1 [==============================] - 0s 13ms/step - loss: 6855.2979 - mean_squared_error: 6855.2979
Epoch 3/2000
1/1 [==============================] - 0s 12ms/step - loss: 6853.9561 - mean_squared_error: 6853.9561
Epoch 4/2000
1/1 [==============================] - 0s 10ms/step - loss: 6852.5962 - mean_squared_error: 6852.5962
Epoch 5/2000
1/1 [==============================] - 0s 10ms/step - loss: 6851.2290 - mean_squared_error: 6851.2290
Epoch 6/2000
1/1 [==============================] - 0s 10ms/step - loss: 6849.8555 - mean_squared_error: 6849.8555
Epoch 7/2000
1/1 [==============================] - 0s 10ms/step - loss: 6848.4746 - mean_squared_error: 6848.4746
Epoch 8/2000
1/1 [==============================] - 0s 16ms/step - loss: 6847.0854 - mean_squared_error: 6847.0854
Epoch 9/2000
1/1 [==============================] - 0s 10ms/step - loss: 6845.7